# Amazon Food Review 요약 실습
- 데이터셋 링크
    https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
    
- 실습개요
• 긴 텍스트를 짧은 요약으로 만들어주는 AI 만들기" Amazon 리뷰 데이터를 사용해서 긴 리뷰 텍스트를 짧은 한 줄 요약으로 바꿔주는 모델을 만드는 실습입니다.

→ 현실에서는

**Hugging Face** 같은 라이브러리 사용

- **미리 훈련된 모델** 활용 (BERT, GPT 등)
- **Fine-tuning**으로 우리 데이터에 맞게 조정 →CNN에서의 전이학습 튜닝

→ 25년도8월에서는  RAG가 대세(에이전트화): 도메인이해, 청크사이즈(어떤 단위로 나뉘어서 기억할것인가?), 효율적인 전처리, 벡터DB이용
  원본소스를 불러오는 과정이 어려움  (원본:텍스트로만, 표로, 이미지텍스트(OCR), pdf, 웹문서, 미술관옆동물원과같은 html문서등) —>파이썬 코드로 해결

.→ 학습결과 비교

Seq2Seq:   10 에포크 → Loss: 0.2826
Attention: 10 에포크 → Loss: 0.2779  (약간 개선)
Transformer: 7 에포크 → Loss: 3.0384 (빠른 수렴)

→ 생성품질비교

Seq2Seq: "Great", "Great", "Great"... (단조로움)
Attention: "Great", "Not for", "Great tea"... (조금 다양)
Transformer: 더 복잡한 패턴 학습 가능 (에포크가 적어서 결과 미완성)

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_utils.py", "helper_utils.py")
import importlib
import helper_utils as hu
importlib.reload(hu)
from helper_utils import *

In [ ]:
# 기본 라이브러리

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import average_precision_score

# --- 기타 라이브러리 ---
from PIL import Image
from PIL import ImageFilter
from PIL import ImageDraw
import albumentations as A
import IPython.display
#from tqdm import tqdm
from tqdm.notebook import tqdm

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as TF
from torch.nn import CrossEntropyLoss
from collections import OrderedDict

# --- 기타 ---
import re
import os
import sys
import json
import math
import random
import yaml
import shutil
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from datetime import datetime
from datetime import timezone, timedelta
import pytz
__kst = pytz.timezone('Asia/Seoul')

# GPU 설정
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

  # 재현 가능한 결과를 위해
np.random.seed(42)
torch.manual_seed(42)
if __device == 'cuda':
    torch.cuda.manual_seed_all(42)

print(f"라이브러리 로드 완료 사용장치:{__device}")

## 데이타 다운로드

In [ ]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("snap/amazon-fine-food-reviews")
print("Path to dataset files:", path)
print_dir_tree(path)

data_csv_path = os.path.join(path, "Reviews.csv")
data_jsonl_path = os.path.join(path, "Reviews.jsonl")


## 데이타 jsonl 변환

In [ ]:
def csv_to_jsonl(input_file, output_file):
    """
    CSV 파일을 읽어 Hugging Face 데이터셋에 적합한 JSON Lines(.jsonl) 형식으로 변환합니다.

    Args:
        input_file (str): 입력 CSV 파일 경로.
        output_file (str): 출력 JSON Lines 파일 경로.
    """
    try:
        # 예시 데이터 로드: io.StringIO를 사용하여 문자열 데이터를 파일처럼 처리
        df = pd.read_csv(input_file)

        # 2. NaN(결측값)을 빈 문자열로 처리 (필요에 따라 선택)
        # 딥러닝 학습 시 빈 값이 오류를 일으킬 수 있으므로 미리 처리합니다.
        df = df.fillna("")

        df.to_json(
            output_file,
            orient='records',
            lines=True,
            force_ascii=False
        )
        print(f"✅ 변환 성공: '{output_file}' 파일이 생성되었습니다.")

    except FileNotFoundError:
        print(f"❌ 오류: 입력 파일 '{input_file}'을 찾을 수 없습니다.")
    except Exception as e:
        print(f"❌ 변환 중 오류 발생: {e}")


if not os.path.exists(data_jsonl_path):
    csv_to_jsonl(data_csv_path, data_jsonl_path)

## 데이타 DataFrame

In [ ]:
from datasets import load_dataset
raw_json = load_dataset("json", data_files=data_jsonl_path)

print(raw_json)
print("-" * 30)
print('raw_datasets', type(raw_json))
print(raw_json['train'][0])

dataset_org = raw_json['train']
print('train_dataset', type(dataset_org))
print('train_dataset[0]', dataset_org[0])

import pandas as pd
df_org = dataset_org.to_pandas()
# 기본정보
print(df_org.shape)
print(df_org.columns.tolist())
print(df_org.isna().sum())
df_org.head_att(3)
df = df_org.copy()


## 데이타 분석 시각화

In [ ]:
# python
# df가 존재한다고 가정 (df = df_org.copy() 상태)
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer

# 기본 복사 안전 처리
df = df_org.copy()

# 타임스탬프 -> datetime, 길이 정보 추가
df['datetime'] = pd.to_datetime(df['Time'], unit='s', errors='coerce')
df['year'] = df['datetime'].dt.year
df['text_char_len'] = df['Text'].astype(str).str.len()
df['text_word_len'] = df['Text'].astype(str).str.split().map(len)
df['summary_char_len'] = df['Summary'].astype(str).str.len()
df['summary_word_len'] = df['Summary'].astype(str).str.split().map(len)

# 출력: 상위 정보 테이블
print("데이터개요:", df.shape)
display(df.head(5))
print("\n컬럼별 결측수:")
print(df.isna().sum())

# Score 분포
plt.figure(figsize=(6,4))
sns.countplot(x='Score', data=df, palette='crest')
plt.title('Score 분포')
plt.show()

# 텍스트/요약 길이 분포 (히스토그램 + 박스플롯)
fig, axes = plt.subplots(2,2, figsize=(12,8))
sns.histplot(df['text_word_len'], bins=50, ax=axes[0,0], kde=True)
axes[0,0].set_title('Text word length 분포')
sns.boxplot(x='Score', y='text_word_len', data=df.sample(20000, random_state=1), ax=axes[0,1])
axes[0,1].set_title('Score별 Text word length (샘플)')

sns.histplot(df['summary_word_len'], bins=40, ax=axes[1,0], kde=True)
axes[1,0].set_title('Summary word length 분포')
sns.boxplot(x='Score', y='summary_word_len', data=df.sample(20000, random_state=2), ax=axes[1,1])
axes[1,1].set_title('Score별 Summary word length (샘플)')
plt.tight_layout()
plt.show()

# 연도별 리뷰 건수 추이 (상위 연도만)
cnt_by_year = df['year'].value_counts().sort_index()
plt.figure(figsize=(8,3))
cnt_by_year.plot(kind='bar')
plt.title('연도별 리뷰 수')
plt.xlabel('year')
plt.show()
print("연도별 리뷰 수 (상위):")
display(cnt_by_year.tail(10))

# Score별 텍스트 길이 통계
print("\nScore별 Text 길이 통계 (word count):")
display(df.groupby('Score')['text_word_len'].agg(['count','mean','median','std']).sort_index())

# 상위 ProductId, UserId
print("\nTop ProductId (리뷰 수 상위 10):")
display(df['ProductId'].value_counts().head(10))
print("\nTop UserId (리뷰 수 상위 10):")
display(df['UserId'].value_counts().head(10))

# 샘플 리뷰 (Score별 1개씩)
print("\nScore별 샘플 Summary / Text:")
for s in sorted(df['Score'].unique()):
    row = df[df['Score']==s].iloc[0]
    print(f"\nScore={s} Summary: {row['Summary'][:120]}")
    print(f"         Text (first 200 chars): {str(row['Text'])[:200]}")

# 텍스트 상위 토큰 (영어 불용어 제외)
cv = CountVectorizer(stop_words='english', max_features=40, token_pattern=r"(?u)\b[a-zA-Z']+\b")
X = cv.fit_transform(df['Text'].astype(str).sample(min(50000, len(df)), random_state=0))
tok_counts = np.array(X.sum(axis=0)).ravel()
tok_idx = tok_counts.argsort()[::-1]
tokens = np.array(cv.get_feature_names_out())[tok_idx]
counts = tok_counts[tok_idx]
top_tokens = pd.DataFrame({'token': tokens, 'count': counts}).head(30)
print("\n상위 토큰 (샘플 50k 문서 기준, stop_words='english'):")
display(top_tokens)

# 상위 Summary 길이/예외 샘플 확인 (짧거나 긴 케이스)
print("\n가장 짧은 Summary 예시:")
display(df.nsmallest(3, 'summary_word_len')[['Score','Summary','Summary','summary_word_len']])
print("\n가장 긴 Summary 예시:")
display(df.nlargest(3, 'summary_word_len')[['Score','Summary','summary_word_len']])

## 데이타 전처리 및 분할

In [ ]:
from datasets import load_dataset, DatasetDict
import re
import html
import numpy as np
import unicodedata

def full_preprocessing_pipeline(jsonl_path):
    """
    강화된 전처리 파이프라인 (리팩토링 버전)
    """
    print("="*60)
    print("Amazon Fine Food Reviews 전처리 시작")
    print("="*60)
    
    # 1. 데이터 로드
    print("\n1. 데이터 로드...")
    raw_datasets = load_dataset("json", data_files=jsonl_path)
    dataset = raw_datasets['train']
    print(f"   원본: {len(dataset):,}개")
    
    # 2. 필요한 컬럼만 선택
    print("\n2. 컬럼 선택 (Text, Summary)...")
    dataset = dataset.select_columns(['Text', 'Summary'])
    
    # 3. 텍스트 정제
    print("\n3. 텍스트 정제...")
    
    def clean_text(text):
        """리팩토링된 텍스트 정제 함수"""
        
        # None 또는 빈 문자열 체크
        if not text or not isinstance(text, str):
            return ""
        
        # ==================== 1. HTML 태그 제거 ====================
        html_patterns = [
            (r'<br\s*/?\s*>', ' ', re.IGNORECASE),  # <br> 변형들
            (r'<[^>]+>', '', 0),                     # 모든 HTML 태그
        ]
        
        for pattern, replacement, flags in html_patterns:
            if flags:
                text = re.sub(pattern, replacement, text, flags=flags)
            else:
                text = re.sub(pattern, replacement, text)
        
        # ==================== 2. HTML 엔티티 디코딩 ====================
        # 중첩된 엔티티 대비 여러 번 수행
        for _ in range(3):
            text = html.unescape(text)
        
        # 남은 HTML 엔티티 강제 제거
        entity_patterns = [
            r'&[a-zA-Z]+;',          # &nbsp; &amp; 등
            r'&#\d+;',               # &#123; 등
            r'&#x[0-9a-fA-F]+;',     # &#xAB; 등
        ]
        
        for pattern in entity_patterns:
            text = re.sub(pattern, '', text)
        
        # ==================== 3. URL/이메일 제거 ====================
        web_patterns = [
            r'http[s]?://\S+',       # http:// https://
            r'www\.\S+',             # www.
            r'\S+@\S+\.\S+',         # 이메일
        ]
        
        for pattern in web_patterns:
            text = re.sub(pattern, '', text)
        
        # ==================== 4. 유니코드 정규화 (Non-ASCII 제거) ====================
        text = unicodedata.normalize('NFKD', text)
        text = text.encode('ascii', 'ignore').decode('ascii')
        
        # ==================== 5. 제어 문자 제거 ====================
        text = ''.join(c if ord(c) >= 32 or c in '\t\n\r' else ' ' for c in text)
        
        # ==================== 6. 소문자 변환 ====================
        text = text.lower()
        
        # ==================== 7. 반복 특수문자 정규화 ====================
        repeated_punct_patterns = [
            (r'\.{2,}', '.'),        # ... → .
            (r'!{2,}', '!'),         # !!! → !
            (r'\?{2,}', '?'),        # ??? → ?
            (r'-{2,}', '-'),         # --- → -
            (r'_{2,}', '_'),         # ___ → _
            (r'={2,}', '='),         # === → =
            (r'\*{2,}', '*'),        # *** → *
            (r'\+{2,}', '+'),        # +++ → +
        ]
        
        for pattern, replacement in repeated_punct_patterns:
            text = re.sub(pattern, replacement, text)
        
        # ==================== 8. 따옴표 정규화 ====================
        quote_replacements = [
            ('"', '"'), ('"', '"'),  # 스마트 쿼트
            (''', "'"), (''', "'"),  # 스마트 어포스트로피
            ('`', "'"), ('´', "'"),  # 백틱, 악센트
        ]
        
        for old, new in quote_replacements:
            text = text.replace(old, new)
        
        # ==================== 9. 대시/하이픈 정규화 ====================
        dash_replacements = [
            ('—', '-'),  # em dash
            ('–', '-'),  # en dash
            ('―', '-'),  # horizontal bar
        ]
        
        for old, new in dash_replacements:
            text = text.replace(old, new)
        
        # ==================== 10. 특수 공백 정규화 ====================
        space_replacements = [
            ('\n', ' '),           # 줄바꿈
            ('\t', ' '),           # 탭
            ('\r', ' '),           # 캐리지 리턴
            ('\xa0', ' '),         # non-breaking space
            ('\u2009', ' '),       # thin space
            ('\u200b', ''),        # zero-width space
            ('\u200c', ''),        # zero-width non-joiner
            ('\u200d', ''),        # zero-width joiner
            ('\ufeff', ''),        # zero-width no-break space (BOM)
        ]
        
        for old, new in space_replacements:
            text = text.replace(old, new)
        
        # ==================== 11. 연속 공백 정규화 ====================
        text = re.sub(r'\s+', ' ', text)
        
        # ==================== 12. 양쪽 공백/특수문자 제거 ====================
        text = text.strip()
        text = text.strip('-._,;:!?')
        text = text.strip()
        
        # ==================== 13. 최종 품질 검증 ====================
        if text and len(text) > 0:
            # 알파벳+공백 비율이 너무 낮으면 버림
            alpha_space_ratio = sum(c.isalpha() or c.isspace() for c in text) / len(text)
            if alpha_space_ratio < 0.5:
                return ""
        
        return text
    
    def preprocess_function(examples):
        texts = [clean_text(t) for t in examples['Text']]
        summaries = [clean_text(s) for s in examples['Summary']]
        text_lengths = [len(t.split()) if t else 0 for t in texts]
        summary_lengths = [len(s.split()) if s else 0 for s in summaries]
        
        return {
            'Text': texts,
            'Summary': summaries,
            'text_length': text_lengths,
            'summary_length': summary_lengths
        }
    
    dataset = dataset.map(
        preprocess_function, 
        batched=True, 
        desc="텍스트 정제",
    )
    
    # 4. 길이 필터링
    print("\n4. 길이 필터링...")
    print(f"   필터링 전: {len(dataset):,}개")
    
    def filter_by_length(example):
        return (10 <= example['text_length'] <= 500 and 
                1 <= example['summary_length'] <= 50)
    
    dataset = dataset.filter(filter_by_length, desc="길이 필터링")
    print(f"   필터링 후: {len(dataset):,}개")
    
    # 5. 품질 필터링
    print("\n5. 품질 필터링...")
    print(f"   필터링 전: {len(dataset):,}개")
    
    def filter_quality(example):
        text = example['Text']
        summary = example['Summary']
        
        # 빈 문자열 체크
        if not text or not summary or text.strip() == '' or summary.strip() == '':
            return False
        
        # 품질 체크 패턴 리스트
        quality_checks = [
            # (체크할 텍스트, 패턴, 설명)
            (text, r'<[^>]+>', 'HTML 태그'),
            (summary, r'<[^>]+>', 'HTML 태그'),
            (text, r'&[a-zA-Z]+;', 'HTML 엔티티'),
            (summary, r'&[a-zA-Z]+;', 'HTML 엔티티'),
        ]
        
        for check_text, pattern, _ in quality_checks:
            if re.search(pattern, check_text):
                return False
        
        # 제어 문자 체크
        for check_text in [text, summary]:
            if any(ord(c) < 32 and c not in '\t\n\r' for c in check_text):
                return False
        
        # 비정상 반복 체크 (같은 단어 5회 연속)
        words = text.split()
        if len(words) >= 5:
            for i in range(len(words) - 4):
                if len(set(words[i:i+5])) == 1:
                    return False
        
        return True
    
    dataset = dataset.filter(filter_quality, desc="품질 필터링")
    print(f"   필터링 후: {len(dataset):,}개")
    
    # 6. 중복 제거
    print("\n6. 중복 제거...")
    print(f"   제거 전: {len(dataset):,}개")
    
    seen = set()
    unique_indices = []
    for idx, text in enumerate(dataset['Text']):
        text_hash = hash(text)
        if text_hash not in seen:
            seen.add(text_hash)
            unique_indices.append(idx)
    
    dataset = dataset.select(unique_indices)
    print(f"   제거 후: {len(dataset):,}개")
    
    # 7. 통계 출력
    print("\n7. 통계:")
    text_lengths = dataset['text_length']
    summary_lengths = dataset['summary_length']
    
    stats = [
        ("Text", text_lengths),
        ("Summary", summary_lengths)
    ]
    
    for name, lengths in stats:
        print(f"   {name} 길이:")
        print(f"      평균: {np.mean(lengths):.1f} 단어")
        print(f"      중앙값: {np.median(lengths):.1f} 단어")
        print(f"      최소: {np.min(lengths)} 단어")
        print(f"      최대: {np.max(lengths)} 단어")
        print(f"      표준편차: {np.std(lengths):.1f}")
        if name == "Text":
            print()
    
    # 8. 데이터 분할
    print("\n8. 데이터 분할...")
    train_val_test = dataset.train_test_split(test_size=0.1, seed=42)
    train_val = train_val_test['train']
    test = train_val_test['test']
    
    train_val_split = train_val.train_test_split(test_size=0.111, seed=42)
    train = train_val_split['train']
    val = train_val_split['test']
    
    print(f"   Train: {len(train):,}개 ({len(train)/len(dataset)*100:.1f}%)")
    print(f"   Val: {len(val):,}개 ({len(val)/len(dataset)*100:.1f}%)")
    print(f"   Test: {len(test):,}개 ({len(test)/len(dataset)*100:.1f}%)")
    
    # 9. 최종 데이터셋
    final_datasets = DatasetDict({
        'train': train,
        'validation': val,
        'test': test
    })
    
    print("\n" + "="*60)
    print("전처리 완료!")
    print("="*60)
    print(f"\n최종 데이터셋:")
    print(final_datasets)
    
    return final_datasets


# 전처리 및 저장
dataset_path = os.path.join(drive_root(), 'data', "AmazonFineFoodReviews.dataset")   

if not os.path.exists(dataset_path):
    processed_datasets = full_preprocessing_pipeline(data_jsonl_path)
    os.makedirs(os.path.dirname(dataset_path), exist_ok=True)
    processed_datasets.save_to_disk(dataset_path)
    print(f"\n저장완료: {dataset_path}")
else:
    print(f"이미 존재하는 데이터셋 경로입니다: {dataset_path}")
    print("기존 데이터를 삭제하고 새로 전처리하려면:")
    print(f"   import shutil")
    print(f"   shutil.rmtree('{dataset_path}')")
    print(f"   # 그 다음 다시 실행")
    final_datasets = DatasetDict.load_from_disk(dataset_path)
    print(final_datasets)

## 분할 데이타 검토

In [ ]:
from datasets import DatasetDict

# 로드
dataset_path = os.path.join(drive_root(), 'data', "AmazonFineFoodReviews.dataset")
final_datasets = DatasetDict.load_from_disk(dataset_path)

print(final_datasets)
print("\n샘플 확인:")
print("-" * 60)

# Train 샘플 3개 출력
for i in range(3):
    sample = final_datasets['train'][i]
    print(f"\n[샘플 {i+1}]")
    print(f"Text ({sample['text_length']} 단어):")
    print(f"  {sample['Text'][:200]}...")
    print(f"\nSummary ({sample['summary_length']} 단어):")
    print(f"  {sample['Summary']}")
    print("-" * 60)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 길이 분포 시각화
train_text_lengths = final_datasets['train']['text_length']
train_summary_lengths = final_datasets['train']['summary_length']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Text 길이 분포
axes[0].hist(train_text_lengths, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Text Length (words)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Text Length Distribution')
axes[0].axvline(np.mean(train_text_lengths), color='r', linestyle='--', 
                label=f'Mean: {np.mean(train_text_lengths):.1f}')
axes[0].legend()

# Summary 길이 분포
axes[1].hist(train_summary_lengths, bins=30, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Summary Length (words)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Summary Length Distribution')
axes[1].axvline(np.mean(train_summary_lengths), color='r', linestyle='--',
                label=f'Mean: {np.mean(train_summary_lengths):.1f}')
axes[1].legend()

plt.tight_layout()
plt.savefig('data_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 시각화 저장 완료: data_distribution.png")

In [ ]:
# ...existing code...
import os, re, pandas as pd
from datasets import load_from_disk

# final_datasets가 없으면 디스크에서 로드
try:
    final_datasets
except NameError:
    dataset_path = os.path.join(drive_root(), 'data', "AmazonFineFoodReviews.dataset")
    final_datasets = load_from_disk(dataset_path)

train = final_datasets['train']
n = len(train)
print(f"Train 샘플 수: {n:,}")

issues = []
html_re = re.compile(r'<[^>]+>')
entity_re = re.compile(r'&[a-zA-Z]+;')

pbar = tqdm(range(n), total=n, desc="샘플 검사 중")
for i in pbar:
    row = train[i]
    text = row.get('Text', '')
    summary = row.get('Summary', '')

    row_issues = []
    # 타입/빈값
    if not isinstance(text, str): row_issues.append('text_not_str')
    if not isinstance(summary, str): row_issues.append('summary_not_str')
    if isinstance(text, str) and text.strip() == '': row_issues.append('text_empty')
    if isinstance(summary, str) and summary.strip() == '': row_issues.append('summary_empty')

    # HTML / 엔티티 잔존
    if isinstance(text, str) and (html_re.search(text) or entity_re.search(text)): row_issues.append('text_html_or_entity')
    if isinstance(summary, str) and (html_re.search(summary) or entity_re.search(summary)): row_issues.append('summary_html_or_entity')

    # 반복된 특수문자(!!!, ???, ...)
    if isinstance(text, str) and re.search(r'([!?\.])\1{2,}', text): row_issues.append('text_repeated_punct')
    if isinstance(summary, str) and re.search(r'([!?\.])\1{2,}', summary): row_issues.append('summary_repeated_punct')

    # 제어문자
    if isinstance(text, str) and any(ord(c) < 32 and c not in '\t\n\r' for c in text): row_issues.append('text_ctrl_char')
    if isinstance(summary, str) and any(ord(c) < 32 and c not in '\t\n\r' for c in summary): row_issues.append('summary_ctrl_char')

    # 길이 이상치 (단어 수)
    if isinstance(text, str):
        tw = len(text.split())
        if tw < 5: row_issues.append(f'text_too_short({tw})')
        if tw > 2000: row_issues.append(f'text_too_long({tw})')
    if isinstance(summary, str):
        sw = len(summary.split())
        if sw < 1: row_issues.append(f'summary_too_short({sw})')
        if sw > 200: row_issues.append(f'summary_too_long({sw})')

    # 비정상 문자(비 ASCII) — 필요에 따라 제거/허용
    if isinstance(text, str) and any(ord(c) > 127 for c in text): row_issues.append('text_non_ascii')
    if isinstance(summary, str) and any(ord(c) > 127 for c in summary): row_issues.append('summary_non_ascii')

    if row_issues:
        issues.append({
            'index': i,
            'text_preview': (text[:200] if isinstance(text,str) else str(text)) ,
            'summary_preview': (summary[:200] if isinstance(summary,str) else str(summary)),
            'issues': ';'.join(row_issues)
        })

print(f"문제 샘플 수: {len(issues):,}")

# 결과 출력 및 저장
df_issues = pd.DataFrame(issues)
display(df_issues.head(50))
# out_path = os.path.join('data', 'train_issues.csv')
# os.makedirs('data', exist_ok=True)
# df_issues.to_csv(out_path, index=False, encoding='utf-8-sig')
# print(f"저장: {out_path}")


In [ ]:
df_issues = pd.DataFrame(issues)
display(df_issues.head(50))


In [ ]:
import random

# 무작위 샘플 10개 확인
random_indices = random.sample(range(len(final_datasets['train'])), 5)

print("무작위 샘플 5개 품질 확인:")
print("=" * 80)

for idx in random_indices:
    sample = final_datasets['train'][idx]
    
    # 품질 체크
    has_special_chars = any(c * 3 in sample['Text'] for c in '!?.') 
    has_html = '&' in sample['Text'] or '<' in sample['Text']
    
    print(f"\n[인덱스: {idx}]")
    print(f"Text: {sample['Text'][:150]}...")
    print(f"Summary: {sample['Summary']}")
    print(f"길이: Text={sample['text_length']}, Summary={sample['summary_length']}")
    
    # 품질 이슈 표시
    issues = []
    if has_special_chars:
        issues.append("특수문자 반복")
    if has_html:
        issues.append("HTML 잔존")
    
    if issues:
        print(f"이슈: {', '.join(issues)}")
    else:
        print("양호")
    
    print("-" * 80)

In [ ]:
from datasets import load_from_disk
from transformers import BartTokenizer
import os

# 데이터 로드
dataset_path = os.path.join(drive_root(), 'data', "AmazonFineFoodReviews.dataset")
final_datasets = load_from_disk(dataset_path)

print(final_datasets)
print(f"\nTrain: {len(final_datasets['train']):,}개")
print(f"Validation: {len(final_datasets['validation']):,}개")
print(f"Test: {len(final_datasets['test']):,}개")

# 셈플10개만

In [ ]:
final_datasets = DatasetDict({
       'train': final_datasets['train'].select(range(10)),
       'validation': final_datasets['validation'].select(range(10)),
       'test': final_datasets['test'].select(range(10))
   })

In [ ]:
# BART 토크나이저 로드
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

def tokenize_function(examples):
    """토크나이제이션 함수"""
    # 입력 (Text)
    model_inputs = tokenizer(
        examples['Text'],
        max_length=200,
        truncation=True,
        padding='max_length'
    )
    
    # 출력 (Summary)
    labels = tokenizer(
        examples['Summary'],
        max_length=25,
        truncation=True,
        padding='max_length'
    )
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 토크나이제이션 적용
print("토크나이제이션 중...")
tokenized_datasets = final_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=['Text', 'Summary', 'text_length', 'summary_length'],
    desc="토크나이제이션"
)

print("\n토크나이제이션 완료!")
print(tokenized_datasets)
print(f"\n샘플 확인:")
print(tokenized_datasets['train'][0])

In [ ]:
from transformers import (
    BartForConditionalGeneration,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)

# 모델 로드
print("BART 모델 로드 중...")
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Data Collator (동적 패딩)
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    logging_dir='./logs',
    logging_steps=500,
    warmup_steps=500,
    fp16=True,  # GPU 있으면 활성화
    report_to="none",  # wandb 비활성화
)

print("\n학습 설정:")
print(f"  배치 크기: {training_args.per_device_train_batch_size}")
print(f"  에폭: {training_args.num_train_epochs}")
print(f"  학습률: {training_args.learning_rate}")

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 학습 시작
print("\n학습 시작...")
print("=" * 60)
trainer.train()

# 모델 저장
model_save_path = "./bart_review_summarizer"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\n모델 저장 완료: {model_save_path}")

In [ ]:
from datasets import load_metric
import numpy as np

# ROUGE 메트릭 로드
rouge_metric = load_metric('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # -100을 패딩 토큰으로 변환
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # 디코딩
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE 계산
    result = rouge_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    
    return {
        'rouge1': result['rouge1'].mid.fmeasure,
        'rouge2': result['rouge2'].mid.fmeasure,
        'rougeL': result['rougeL'].mid.fmeasure,
    }

# 테스트 데이터 평가
print("\n테스트 데이터 평가 중...")
test_results = trainer.predict(tokenized_datasets['test'])
print("\n평가 결과:")
print(f"  ROUGE-1: {test_results.metrics.get('test_rouge1', 0):.4f}")
print(f"  ROUGE-2: {test_results.metrics.get('test_rouge2', 0):.4f}")
print(f"  ROUGE-L: {test_results.metrics.get('test_rougeL', 0):.4f}")

In [ ]:
def generate_summary(text, max_length=25):
    """리뷰 요약 생성"""
    # 토크나이징
    inputs = tokenizer(
        text,
        max_length=200,
        truncation=True,
        return_tensors='pt'
    )
    
    # GPU 사용 시
    if torch.cuda.is_available():
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        model.to('cuda')
    
    # 생성
    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_beams=4,
        length_penalty=0.8,
        early_stopping=True,
        no_repeat_ngram_size=3
    )
    
    # 디코딩
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 테스트
test_text = """
I absolutely love this coffee! The flavor is rich and smooth, 
not bitter at all. The aroma is amazing and fills the entire kitchen. 
The packaging keeps it fresh for a long time. 
Great value for the price. Highly recommend to all coffee lovers!
"""

summary = generate_summary(test_text)
print(f"입력: {test_text[:150]}...")
print(f"\n생성된 요약: {summary}")

# 여러 샘플 테스트
print("\n" + "=" * 60)
print("테스트 샘플 3개:")
print("=" * 60)

for i in range(3):
    sample = final_datasets['test'][i]
    generated = generate_summary(sample['Text'])
    print(f"\n[샘플 {i+1}]")
    print(f"원본: {sample['Text'][:150]}...")
    print(f"실제 요약: {sample['Summary']}")
    print(f"생성 요약: {generated}")
    print("-" * 60)

In [ ]:
from datasets import load_from_disk, DatasetDict
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
import os

# ==================== 1. 데이터 로드 및 샘플링 ====================
dataset_path = os.path.join(drive_root(), 'data', "AmazonFineFoodReviews.dataset")
full_datasets = load_from_disk(dataset_path)

print("원본 데이터:")
print(full_datasets)
print(f"\nTrain: {len(full_datasets['train']):,}개")
print(f"Validation: {len(full_datasets['validation']):,}개")
print(f"Test: {len(full_datasets['test']):,}개")

# 테스트용 샘플링 (10개씩)
print("\n" + "="*60)
print("테스트용 샘플링: 각 10개씩")
print("="*60)

final_datasets = DatasetDict({
    'train': full_datasets['train'].select(range(10)),
    'validation': full_datasets['validation'].select(range(10)),
    'test': full_datasets['test'].select(range(10))
})

print(f"\nTrain: {len(final_datasets['train'])}개")
print(f"Validation: {len(final_datasets['validation'])}개")
print(f"Test: {len(final_datasets['test'])}개")

# 샘플 확인
print("\n샘플 데이터 확인:")
sample = final_datasets['train'][0]
print(f"Text: {sample['Text'][:100]}...")
print(f"Summary: {sample['Summary']}")

# ==================== 2. 토크나이저 및 토크나이제이션 ====================
print("\n" + "="*60)
print("토크나이제이션")
print("="*60)

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

def tokenize_function(examples):
    """토크나이제이션 함수"""
    # 입력
    model_inputs = tokenizer(
        examples['Text'],
        max_length=200,
        truncation=True,
        padding='max_length'
    )
    
    # 출력
    labels = tokenizer(
        examples['Summary'],
        max_length=25,
        truncation=True,
        padding='max_length'
    )
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 토크나이제이션 적용
tokenized_datasets = final_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=['Text', 'Summary', 'text_length', 'summary_length'],
    desc="토크나이제이션"
)

print("토크나이제이션 완료!")
print(tokenized_datasets)

# ==================== 3. 모델 로드 ====================
print("\n" + "="*60)
print("모델 로드")
print("="*60)

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
print("BART 모델 로드 완료!")

# Data Collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

# ==================== 4. 학습 설정 (테스트용) ====================
print("\n" + "="*60)
print("학습 설정 (테스트용)")
print("="*60)

training_args = TrainingArguments(
    output_dir='./results_test',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=2,  # 작은 배치
    per_device_eval_batch_size=2,
    num_train_epochs=1,  # 1 에폭만
    weight_decay=0.01,
    save_total_limit=1,
    logging_steps=2,  # 자주 로깅
    warmup_steps=2,
    fp16=False,  # CPU 호환
    report_to="none",
)

print("학습 설정:")
print(f"  데이터: Train {len(tokenized_datasets['train'])}개")
print(f"  배치 크기: {training_args.per_device_train_batch_size}")
print(f"  에폭: {training_args.num_train_epochs}")
print(f"  학습률: {training_args.learning_rate}")

# ==================== 5. Trainer 및 학습 ====================
print("\n" + "="*60)
print("학습 시작")
print("="*60)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 학습
trainer.train()

print("\n학습 완료!")

# ==================== 6. 추론 테스트 ====================
print("\n" + "="*60)
print("추론 테스트")
print("="*60)

import torch

def generate_summary(text, max_length=25):
    """요약 생성"""
    inputs = tokenizer(
        text,
        max_length=200,
        truncation=True,
        return_tensors='pt'
    )
    
    # GPU 사용 가능 시
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 생성
    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_beams=4,
        length_penalty=0.8,
        early_stopping=True,
        no_repeat_ngram_size=3
    )
    
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


# # 테스트 샘플 3개
# print("\n테스트 샘플 생성:")
# print("="*60)

# for i in range(min(3, len(final_datasets['test']))):
#     sample = final_datasets['test'][i]
#     generated = generate_summary(sample['Text'])
    
#     print(f"\n[샘플 {i+1}]")
#     print(f"입력: {sample['Text'][:100]}...")
#     print(f"실제 요약: {sample['Summary']}")
#     print(f"생성 요약: {generated}")
#     print("-" * 60)

# print("\n✅ 파이프라인 테스트 완료!")


원본 데이터:
DatasetDict({
    train: Dataset({
        features: ['Text', 'Summary', 'text_length', 'summary_length'],
        num_rows: 312979
    })
    validation: Dataset({
        features: ['Text', 'Summary', 'text_length', 'summary_length'],
        num_rows: 39079
    })
    test: Dataset({
        features: ['Text', 'Summary', 'text_length', 'summary_length'],
        num_rows: 39118
    })
})

Train: 312,979개
Validation: 39,079개
Test: 39,118개

테스트용 샘플링: 각 10개씩

Train: 10개
Validation: 10개
Test: 10개

샘플 데이터 확인:
Text: do not buy this! my cats wouldn't eat it, and within two days, i figured out why. i took it out of t...
Summary: worst product ever

토크나이제이션
토크나이제이션 완료!
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],


Epoch,Training Loss,Validation Loss
1,11.803600,10.661615



학습 완료!

추론 테스트

테스트 샘플 생성:

[샘플 1]
입력: green mountain nantucket blend is a palate pleasing mild-medium blend. i would put it more towards t...
실제 요약: more mild than medium. smooth. no aftertaste
생성 요약: green mountain nantucket blend is a palate pleasing mild-medium blend. i would put it more towards the
------------------------------------------------------------

[샘플 2]
입력: i was a little worried my cat might not like this food after reading some of the reviews. i bought t...
실제 요약: my cat loves it
생성 요약: i was a little worried my cat might not like this food after reading some of the reviews. i bought the
------------------------------------------------------------

[샘플 3]
입력: hugs, 12-ounce bags (pack of 4) was fresh, very good candy.but i hate to admitt this but they are al...
실제 요약: very good
생성 요약: hugs, 12-ounce bags (pack of 4) was fresh, very good candy.but i hate
------------------------------------------------------------

✅ 파이프라인 테스트 완료!


In [ ]:
!pip -q install googletrans==4.0.0-rc1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 9.6 MB/s  0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17518 sha256=1ab1f69f490bd60df2ef4d9b0dbff2632b4f41b82609b1197913194054a867ce
  Stored in directory: c:\users\sw1\appdata\local\pip\cache\wheels\c0\59\9f\7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans

   ----------------------------------------  0/11 [rfc3986]
  Attempting uninstall: hyperframe
   ----------------------------------------  0/11 [rfc3986]
    Found existing installation: hyperframe 6.1.0
   ----------------------------------------  0/11 [rfc3986]
    Uninstalling hyperframe-6.1.0:
   ----------------------------------------  0/11 [rfc3986]
      Successfully uninstalled hyperframe-6.1.0
   -----------------------

  DEPRECATION: Building 'googletrans' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'googletrans'. Discussion can be found at https://github.com/pypa/pip/issues/6334
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.31.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.13.3 which is incompatible.
gradio 5.43.1 requires httpx<1.0,>=0.24.1, but you have httpx 0.13.3 which is incompatible.
gradio-client 1.12.1 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.
mcp 1.13.1 requires httpx>=0.27.1, but you have httpx 0.13.3 which is inc

In [115]:
# 설치 (처음 한 번만)
# !pip install googletrans==4.0.0-rc1

from googletrans import Translator
import time

# 번역기 초기화
translator = Translator()

def translate_to_korean(text):
    """영문을 한글로 번역"""
    try:
        result = translator.translate(text, src='en', dest='ko')
        time.sleep(0.5)  # API 제한 방지
        return result.text
    except Exception as e:
        return f"[번역 실패: {str(e)}]"

# ==================== 테스트 샘플 생성 (번역 추가) ====================
print("\n테스트 샘플 생성:")
print("="*60)

for i in range(min(3, len(final_datasets['test']))):
    sample = final_datasets['test'][i]
    generated = generate_summary(sample['Text'])
    
    print(f"\n[샘플 {i+1}]")
    print("-" * 60)
    
    # 영문 출력
    print("📝 입력 (영문):")
    print(f"  {sample['Text'][:150]}...")
    print(f"\n✅ 실제 요약 (영문): {sample['Summary']}")
    print(f"🤖 생성 요약 (영문): {generated}")
    
    # 한글 번역
    print("\n" + "="*60)
    print("🇰🇷 한글 번역:")
    print("="*60)
    
    input_ko = translate_to_korean(sample['Text'][:150])
    print(f"📝 입력 (한글):")
    print(f"  {input_ko}...")
    
    actual_ko = translate_to_korean(sample['Summary'])
    print(f"\n✅ 실제 요약 (한글): {actual_ko}")
    
    generated_ko = translate_to_korean(generated)
    print(f"🤖 생성 요약 (한글): {generated_ko}")
    
    print("-" * 60)

print("\n파이프라인 테스트 완료!")


테스트 샘플 생성:

[샘플 1]
------------------------------------------------------------
📝 입력 (영문):
  green mountain nantucket blend is a palate pleasing mild-medium blend. i would put it more towards the mild side than the medium side. it is very smoo...

✅ 실제 요약 (영문): more mild than medium. smooth. no aftertaste
🤖 생성 요약 (영문): green mountain nantucket blend is a palate pleasing mild-medium blend. i would put it more towards the

🇰🇷 한글 번역:
📝 입력 (한글):
  Green Mountain Nantucket Blend는 구개를 유쾌한 온화한 중간 조합입니다.나는 그것을 중간면보다 온화한쪽으로 더 넣을 것입니다.매우 스모입니다...

✅ 실제 요약 (한글): 중간보다 온화합니다.매끄러운.뒷맛이 없습니다
🤖 생성 요약 (한글): Green Mountain Nantucket Blend는 구개를 유쾌한 온화한 중간 조합입니다.나는 그것을 더 많이 넣을 것이다
------------------------------------------------------------

[샘플 2]
------------------------------------------------------------
📝 입력 (영문):
  i was a little worried my cat might not like this food after reading some of the reviews. i bought the turkey and salmon formula and this is the best ...

✅ 실제 요약 (영문): my cat loves it
🤖 